In [121]:
import pandas as pd
import numpy as np
from math import sqrt
import pickle
import requests
from pandas import json_normalize
import json

#from io import BytesIO
#from io import StringIO
#mport boto3

import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import date
from datetime import datetime
import datetime
random_state = 5

import urllib.request
from html_table_parser.parser import HTMLTableParser
from pprint import pprint

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, confusion_matrix
import shap

from statsmodels.stats.outliers_influence import variance_inflation_factor

#adjust view
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)

## updating historical predictions file, creating placeholder file containing the previous weeks games in order to scrape stats later

In [2]:
historical = pd.read_csv('data/agg_team_stats.csv')
probs_historical = pd.read_csv('predictions/model_prediction_file.csv')
historical = historical.drop(columns='Unnamed: 0')
probs_historical = probs_historical.drop(columns='Unnamed: 0')

## declaring model name that I would like to use
filename = 'models/random_forest_model_1.sav'
# load the model from disk
model_obj = pickle.load(open(filename, 'rb'))

In [127]:
def get_current_games():
    url = "https://site.web.api.espn.com/apis/v2/scoreboard/header?sport=football&league=nfl&region=us&lang=en&contentorigin=espn&buyWindow=1m&showAirings=buy%2Clive%2Creplay&showZipLookup=true&tz=America/New_York"

    payload={}
    headers = {
    'Accept': 'application/json',
    'Cookie': 'SWID=3750444F-BA06-4929-C99C-62333D7EE8A0'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    response_text = json.loads(response.text)

    game_df = json_normalize(response_text['sports'], ['leagues', 'events'])
    game_dict = game_df.to_dict('index')
    return game_df, game_dict

In [128]:
game_df, game_dict = get_current_games()
game_df[['away','home']] = game_df.shortName.apply(lambda x: pd.Series(str(x).split("@")))
game_df['away'] = game_df['away'].str.strip()
game_df['home'] = game_df['home'].str.strip()
game_df[['date','time']] = game_df.date.apply(lambda x: pd.Series(str(x).split("T")))
game_df['time'] = game_df['time'].str[:5]
game_df['time'] = game_df['time'].str.replace("18", "12")
game_df['time'] = game_df['time'].str.replace("21", "03")
game_df['time'] = game_df['time'].str.replace("01", "07")
game_df

,gamecastAvailable,playByPlayAvailable,commentaryAvailable,recent,id,...,odds.away.moneyLine,airings,away,home,time
0,False,False,False,False,401326526,...,125,NaN,BAL,CLE,12:00
1,False,False,False,False,401326530,...,350,NaN,JAX,TEN,12:00
2,False,False,False,False,401326528,...,350,NaN,LV,KC,12:00
3,False,False,False,False,401326529,...,-230,NaN,NO,NYJ,12:00
4,False,False,False,False,401326531,...,-280,NaN,DAL,WSH,12:00
5,False,False,False,False,401326524,...,120,NaN,ATL,CAR,12:00
6,False,False,False,False,401326527,...,-430,NaN,SEA,HOU,12:00
7,False,False,False,False,401326532,...,425,NaN,DET,DEN,03:05
8,False,False,False,False,401326533,...,340,NaN,NYG,LAC,03:05
9,False,False,False,False,401326525,...,-130,NaN,SF,CIN,03:25


## create file consisting of the current games this week. We will create a file named "previous_weeks_games.csv" this is intended to have an easy way to know which games we will need to go update with stats later

In [136]:
previous_week_games_file_df = game_df[['id', 'season', 'seasonType', 'week']]
previous_week_games_file_df['season'] = previous_week_games_file_df['season'].astype(str)
previous_week_games_file_df['week'] = previous_week_games_file_df['week'].astype(str)
previous_week_games_file_df['url'] = "https://www.espn.com/nfl/scoreboard/_/week/" + previous_week_games_file_df['week'] + "/year/" + previous_week_games_file_df['season'] + "/seasontype/" + previous_week_games_file_df['seasonType']
previous_week_games_file_df = previous_week_games_file_df.drop(columns = ['season', 'seasonType', 'week'])
previous_week_games_file_df.head()



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,url
0,401326526,https://www.espn.com/nfl/scoreboard/_/week/14/...
1,401326530,https://www.espn.com/nfl/scoreboard/_/week/14/...
2,401326528,https://www.espn.com/nfl/scoreboard/_/week/14/...
3,401326529,https://www.espn.com/nfl/scoreboard/_/week/14/...
4,401326531,https://www.espn.com/nfl/scoreboard/_/week/14/...


In [137]:
previous_week_games_file_df.to_csv('predictions/previous_week_games.csv')

In [7]:
#id_list = []
probabilities_df = pd.DataFrame()

for ind in game_df.index:

    game_id = game_df['id'][ind]
    #id_list.append(game_id)

    away = game_df['away'][ind]
    home = game_df['home'][ind]

    away_stat = game_df[game_df['away'] == away]
    away_stat = pd.DataFrame(away_stat['away'])
    away_stat = away_stat.rename(columns = {'away': 'team'})
    #away_stat.columns = ['team']
    away_stat = away_stat.merge(historical, on='team', how='inner')
    away_stat = away_stat.rename(columns = {'first_downs':'first_downs_away', 'first_downs_passing':'first_downs_passing_away', 'first_down_rushing':'first_down_rushing_away', 'first_down_by_penalty':'first_down_by_penalty_away', 'total_plays':'total_plays_away', 'total_yards':'total_yards_away', 'total_drives':'total_drives_away', 'yards_per_play':'yards_per_play_away', 'passing':'passing_away', 'yards_per_pass':'yards_per_pass_away', 'int_thrown':'int_thrown_away', 'rushing':'rushing_away', 'rushing_att':'rushing_att_away', 'yards_per_rush':'yards_per_rush_away', 'turnovers':'turnovers_away', 'fumbles_lost':'fumbles_lost_away', 'defensive_td':'defensive_td_away', 'possession':'possession_away', 'sack_yards_lost_occur':'sack_yards_lost_away_occur', 'sack_yards_lost_yards':'sack_yards_lost_away_yards', 'penalty_occur':'penalty_away_occur', 'penalty_yards':'penalty_away_yards'})
    away_stat = away_stat.reset_index()
    away_stat = away_stat.drop(columns = ['index', 'team'])

    home_stat = game_df[game_df['home'] == home]
    home_stat = pd.DataFrame(home_stat['home'])
    home_stat = home_stat.rename(columns = {'home': 'team'})
    #home_stat.columns = ['team']
    home_stat = home_stat.merge(historical, on='team', how='inner')
    home_stat = home_stat.rename(columns = {'first_downs':'first_downs_home', 'first_downs_passing':'first_downs_passing_home', 'first_down_rushing':'first_down_rushing_home', 'first_down_by_penalty':'first_down_by_penalty_home', 'total_plays':'total_plays_home', 'total_yards':'total_yards_home', 'total_drives':'total_drives_home', 'yards_per_play':'yards_per_play_home', 'passing':'passing_home', 'yards_per_pass':'yards_per_pass_home', 'int_thrown':'int_thrown_home', 'rushing':'rushing_home', 'rushing_att':'rushing_att_home', 'yards_per_rush':'yards_per_rush_home', 'turnovers':'turnovers_home', 'fumbles_lost':'fumbles_lost_home', 'defensive_td':'defensive_td_home', 'possession':'possession_home', 'sack_yards_lost_occur':'sack_yards_lost_home_occur', 'sack_yards_lost_yards':'sack_yards_lost_home_yards', 'penalty_occur':'penalty_home_occur', 'penalty_yards':'penalty_home_yards'})
    home_stat = home_stat.reset_index()
    home_stat = home_stat.drop(columns = ['index', 'team'])  

    stats_all = pd.concat([away_stat, home_stat], axis=1)

    prediction_temp = model_obj.predict(stats_all)
    #prediction_value = str(prediction_temp[0])
    #predictions_list = predictions_list.append([prediction_value])
    probabilities_temp = pd.DataFrame(model_obj.predict_proba(stats_all), columns=['away', 'home'])
    probabilities_temp['id'] = game_id
    #probabilities_temp = probabilities_temp.rename(columns = {'away': away, 'home': home})
    probabilities_df = probabilities_df.append(probabilities_temp)

In [8]:
#probabilities_df.to_csv('predictions/previous_week_games.csv')

In [9]:
probs_historical

,away,home,id
0,0.289347,0.710653,401326494
1,0.559391,0.440609,401326495
2,0.390406,0.609594,401326496
3,0.430924,0.569076,401326497
4,0.420904,0.579096,401326499
5,0.462422,0.537578,401326501
6,0.413134,0.586866,401326502
7,0.483659,0.516341,401326503
8,0.214594,0.785406,401326500
9,0.281802,0.718198,401326498


In [12]:
probs_historical = probs_historical.append(probabilities_df)
#probs_historical

In [14]:
probs_historical.to_csv('predictions/model_prediction_file.csv')

## scrape stats for last weeks games and create meta data files

In [30]:
previous_weeks_games = pd.read_excel('predictions/game_stats_catch_up.xlsx')
#previous_weeks_games = pd.read_csv('predictions/previous_week_games.csv')
previous_weeks_games = previous_weeks_games.drop(columns = ['Unnamed: 0'])
previous_weeks_games['id'] = previous_weeks_games['id'].astype(str)
previous_weeks_games['id'] = previous_weeks_games['id'].str[:-2]
previous_weeks_games_list = previous_weeks_games['id']
previous_weeks_games_list

0     401326494
1     401326495
2     401326496
3     401326497
4     401326499
5     401326501
6     401326502
7     401326503
8     401326500
9     401326498
10    401326504
11    401326505
12    401326506
13    401326507
14    401326508
15    401326509
16    401326510
17    401326511
18    401326512
19    401326513
20    401326516
21    401326517
22    401326514
23    401326518
24    401326519
25    401326520
26    401326521
27    401326515
28    401326522
Name: id, dtype: object

In [31]:
# Opens a website and read its
# binary contents (HTTP Response Body)
def url_get_contents(url):

    # Opens a website and read its
    # binary contents (HTTP Response Body)

    #making request to the website
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)

    #reading contents of the website
    return f.read()

In [35]:
game_stats_big_df = pd.DataFrame()

for i in previous_weeks_games_list:
    
    try:
        web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

        # defining the html contents of a URL.
        xhtml = url_get_contents(web).decode('utf-8')

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)

        # Now finally obtaining the data of
        # the table required
        df_testing = pd.DataFrame(p.tables[1])
        column_list = list(df_testing[0].values)

        away = pd.DataFrame(df_testing[1]).T
        away.columns = column_list
        away = away.reset_index()
        away = away.drop(columns = ['index', 'Matchup'])
        #away.drop(away.columns[[13]], axis=1, inplace=True)
        away = away.rename(columns = {'1st Downs': 'first_downs_away',
                            'Passing 1st downs': 'first_downs_passing_away',
                            'Rushing 1st downs': 'first_downs_rushing_away', 
                            '1st downs from penalties': 'first_down_by_penalty_away', 
                            '3rd down efficiency': 'third_down_away', 
                            '4th down efficiency': 'fourth_down_away', 
                            'Total Plays': 'total_plays_away', 
                            'Total Yards': 'total_yards_away', 
                            'Total Drives': 'total_drives_away', 
                            'Yards per Play': 'yards_per_play_away', 
                            'Passing': 'passing_away', 
                            'Comp-Att': 'comp_att_away', 
                            'Yards per pass': 'yards_per_pass_away',
                            'Fumbles lost': 'fumbles_lost_away',  
                            'Sacks-Yards Lost': 'sack_yards_lost_away', 
                            'Rushing': 'rushing_away', 
                            'Rushing Attempts': 'rushing_att_away', 
                            'Yards per rush': 'yards_per_rush_away', 
                            'Red Zone (Made-Att)': 'red_zone_att_away', 
                            'Penalties': 'penalty_away', 
                            'Turnovers': 'turnovers_away', 
                            'Interceptions thrown': 'int_thrown_away', 
                            'Rushing 1st downs': 'first_down_rushing_away', 
                            'Defensive / Special Teams TDs': 'defensive_td_away', 
                            'Possession': 'possession_away'})

        home = pd.DataFrame(df_testing[2]).T
        home.columns = column_list
        home = home.reset_index()
        home = home.drop(columns = ['index', 'Matchup'])
        #home.drop(home.columns[[13]], axis=1, inplace=True)
        home = home.rename(columns = {'1st Downs': 'first_downs_home',
                            'Passing 1st downs': 'first_downs_passing_home',
                            'Rushing 1st downs': 'first_downs_rushing_home', 
                            '1st downs from penalties': 'first_down_by_penalty_home', 
                            '3rd down efficiency': 'third_down_home', 
                            '4th down efficiency': 'fourth_down_home', 
                            'Total Plays': 'total_plays_home', 
                            'Total Yards': 'total_yards_home', 
                            'Total Drives': 'total_drives_home', 
                            'Yards per Play': 'yards_per_play_home', 
                            'Passing': 'passing_home', 
                            'Comp-Att': 'comp_att_home', 
                            'Yards per pass': 'yards_per_pass_home',
                            'Fumbles lost': 'fumbles_lost_home',  
                            'Sacks-Yards Lost': 'sack_yards_lost_home', 
                            'Rushing': 'rushing_home', 
                            'Rushing Attempts': 'rushing_att_home', 
                            'Yards per rush': 'yards_per_rush_home', 
                            'Red Zone (Made-Att)': 'red_zone_att_home', 
                            'Penalties': 'penalty_home', 
                            'Turnovers': 'turnovers_home', 
                            'Interceptions thrown': 'int_thrown_home', 
                            'Rushing 1st downs': 'first_down_rushing_home', 
                            'Defensive / Special Teams TDs': 'defensive_td_home', 
                            'Possession': 'possession_home'})

        game_stat_temp = pd.concat([away, home], axis=1)
        game_stat_temp['id'] = i
        game_stats_big_df = game_stats_big_df.append(game_stat_temp)

    except Exception as e:
        print("Error:", e)

In [36]:
game_stats_big_df

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,third_down_away,...,fumbles_lost_home,int_thrown_home,defensive_td_home,possession_home,id
0,21,18,3,0,5-13,...,1,0,0,27:04,401326494
0,28,14,7,7,3-13,...,0,0,1,26:30,401326495
0,21,11,9,1,8-13,...,0,1,0,25:22,401326496
0,16,15,1,0,7-13,...,0,1,1,35:20,401326497
0,28,11,13,4,2-8,...,3,2,0,30:01,401326499
0,10,5,5,0,4-14,...,1,0,1,37:34,401326501
0,18,6,11,1,5-11,...,0,0,0,28:58,401326502
0,19,8,11,0,4-11,...,0,0,0,32:02,401326503
0,21,12,6,3,5-12,...,1,1,0,29:08,401326500
0,18,6,10,2,2-13,...,0,1,0,26:18,401326498


## Scrape Previous Week Games Results

In [37]:
game_score_big_df = pd.DataFrame()

for i in previous_weeks_games_list:
    
    try:
        # defining the html contents of a URL.
        web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

        xhtml = url_get_contents(web).decode('utf-8')

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)

        df_score = pd.DataFrame(p.tables[0])
        df_score = pd.DataFrame(df_score.iloc[1: , :])
        team_column = df_score[0].to_frame()
        #team_column = team_column.rename(columns = {0: 'team'})
        #team_df = pd.DataFrame(team_column, columns = ['team'])
        score_column = df_score.iloc[: , -1].to_frame()
        #score_column = score_column.rename({: 'score'})
        #score_df = pd.DataFrame(score_column, columns = ['score'])
        df_score_new = pd.concat([team_column, score_column], axis=1)
        df_score_new.columns = ['team', 'score']

        away_score = pd.DataFrame(df_score_new.iloc[0]).T
        away_score = away_score.reset_index()
        away_score = away_score.drop(columns = ['index'])
        #away_score = away_score[['team', 'score']]
        away_score = away_score.rename(columns = {'team': 'away_team', 'score': 'away_score'})

        home_score = pd.DataFrame(df_score_new.iloc[1]).T
        home_score = home_score.reset_index()
        home_score = home_score.drop(columns = ['index'])
        #home_score = home_score[['team', 'score']]
        home_score = home_score.rename(columns = {'team': 'home_team', 'score': 'home_score'})

        game_score_temp = pd.concat([away_score, home_score], axis=1)
        game_score_temp['id'] = i
        #game_score_temp = game_score_temp.reset_index()
        #game_score_temp = game_score_temp.drop(columns = ['index'])
        game_score_big_df = game_score_big_df.append(game_score_temp)

    except Exception as e:
        print('Error:', e, 'game_id=', i)

In [38]:
game_score_big_df

,away_team,away_score,home_team,home_score,id
0,CHI,16,DET,14,401326494
0,LV,36,DAL,33,401326495
0,BUF,31,NO,6,401326496
0,PIT,10,CIN,41,401326497
0,TB,38,IND,31,401326499
0,CAR,10,MIA,33,401326501
0,TEN,13,NE,36,401326502
0,PHI,7,NYG,13,401326503
0,ATL,21,JAX,14,401326500
0,NYJ,21,HOU,14,401326498


## Merge Game Stats & Game Results

In [83]:
df_ready_to_clean = game_stats_big_df.merge(game_score_big_df, on='id', how='inner')
df_ready_to_clean.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,third_down_away,...,id,away_team,away_score,home_team,home_score
0,21,18,3,0,5-13,...,401326494,CHI,16,DET,14
1,28,14,7,7,3-13,...,401326495,LV,36,DAL,33
2,21,11,9,1,8-13,...,401326496,BUF,31,NO,6
3,16,15,1,0,7-13,...,401326497,PIT,10,CIN,41
4,28,11,13,4,2-8,...,401326499,TB,38,IND,31


In [84]:
df_ready_to_clean.shape

(29, 55)

## cleaning dataset for model training and historical stats file

In [85]:
def convert_fraction(df, column_name, new_column_name):
    # new data frame with split value columns
    new = df[column_name].str.split("-", n = 1, expand = True)
  
    # making separate first name column from new data frame
    df["numerator"]= new[0]
    df["numerator"] = df["numerator"].astype(int)
  
    # making separate last name column from new data frame
    df["denominator"]= new[1]
    df["denominator"] = df["denominator"].astype(int)

    df[new_column_name] = df["numerator"]/df["denominator"]

    # Dropping old Name columns
    df.drop(columns =[column_name, 'numerator', 'denominator'], inplace = True)
    return df

In [86]:
df_ready_to_clean = convert_fraction(df_ready_to_clean, "third_down_away", "third_down_away")
df_ready_to_clean = convert_fraction(df_ready_to_clean, "third_down_home", "third_down_home")
df_ready_to_clean = convert_fraction(df_ready_to_clean, "fourth_down_away", "fourth_down_away")
df_ready_to_clean = convert_fraction(df_ready_to_clean, "fourth_down_home", "fourth_down_home")
df_ready_to_clean = convert_fraction(df_ready_to_clean, "comp_att_away", "comp_att_away")
df_ready_to_clean = convert_fraction(df_ready_to_clean, "comp_att_home", "comp_att_home")
df_ready_to_clean = df_ready_to_clean.fillna(0)
df_ready_to_clean.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,id,away_team,away_score,home_team,home_score
0,21,18,3,0,69,...,401326494,CHI,16,DET,14
1,28,14,7,7,77,...,401326495,LV,36,DAL,33
2,21,11,9,1,62,...,401326496,BUF,31,NO,6
3,16,15,1,0,59,...,401326497,PIT,10,CIN,41
4,28,11,13,4,63,...,401326499,TB,38,IND,31


In [87]:
def convert_att_yards(df, column_name):
    # new data frame with split value columns
    new = df[column_name].str.split("-", n = 1, expand = True)
  
    # making separate first name column from new data frame
    att_col_str = column_name + "_occur"
    yards_col_str = column_name + "_yards"

    df[att_col_str]= new[0]
    df[att_col_str] = df[att_col_str].astype(int)
  
    # making separate last name column from new data frame
    df[yards_col_str]= new[1]
    df[yards_col_str] = df[yards_col_str].astype(int)

    # Dropping old Name columns
    df.drop(columns =[column_name], inplace = True)
    return df

In [88]:
## need to clean a few other variables that are being treated as str's
df_ready_to_clean = convert_att_yards(df_ready_to_clean, "sack_yards_lost_away")
df_ready_to_clean = convert_att_yards(df_ready_to_clean, "sack_yards_lost_home")
df_ready_to_clean = convert_att_yards(df_ready_to_clean, "penalty_away")
df_ready_to_clean = convert_att_yards(df_ready_to_clean, "penalty_home")

## change possession into float
df_ready_to_clean['possession_away'] = df_ready_to_clean['possession_away'].str.replace(':', '.')
df_ready_to_clean['possession_home'] = df_ready_to_clean['possession_home'].str.replace(':', '.')
df_ready_to_clean['possession_away'] = df_ready_to_clean['possession_away'].astype(float)
df_ready_to_clean['possession_home'] = df_ready_to_clean['possession_home'].astype(float)
df_ready_to_clean.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,sack_yards_lost_home_yards,penalty_away_occur,penalty_away_yards,penalty_home_occur,penalty_home_yards
0,21,18,3,0,69,...,8,5,50,10,67
1,28,14,7,7,77,...,2,14,110,14,166
2,21,11,9,1,62,...,17,4,35,4,24
3,16,15,1,0,59,...,18,4,30,3,18
4,28,11,13,4,63,...,21,4,44,6,66


In [89]:
## needing to declare the winning team to create target variable
df_ready_to_clean['target'] = np.where(df_ready_to_clean['away_score'] > df_ready_to_clean['home_score'], 1, 0)
df_ready_to_clean['winning_team'] = np.where(df_ready_to_clean['away_score'] > df_ready_to_clean['home_score'], df_ready_to_clean['away_team'], df_ready_to_clean['home_team'])
df_ready_to_clean.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_away_yards,penalty_home_occur,penalty_home_yards,target,winning_team
0,21,18,3,0,69,...,50,10,67,1,CHI
1,28,14,7,7,77,...,110,14,166,1,LV
2,21,11,9,1,62,...,35,4,24,0,NO
3,16,15,1,0,59,...,30,3,18,0,CIN
4,28,11,13,4,63,...,44,6,66,1,TB


In [90]:
## dropping a few last un-needed cloumns
df_ready_to_clean = df_ready_to_clean.loc[:,~df_ready_to_clean.columns.duplicated()]
df_ready_to_clean = df_ready_to_clean.merge(previous_week_games_file_df, on='id', how='inner')
df_clean = df_ready_to_clean.drop(columns=['red_zone_att_away', 'red_zone_att_home'])
#df_clean.to_csv('data/nfl_historical_clean.csv')
print(df_clean.groupby('target').size())
df_clean.info()

target
0    12
1    17
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 28
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   first_downs_away            29 non-null     object 
 1   first_downs_passing_away    29 non-null     object 
 2   first_down_rushing_away     29 non-null     object 
 3   first_down_by_penalty_away  29 non-null     object 
 4   total_plays_away            29 non-null     object 
 5   total_yards_away            29 non-null     object 
 6   total_drives_away           29 non-null     object 
 7   yards_per_play_away         29 non-null     object 
 8   passing_away                29 non-null     object 
 9   yards_per_pass_away         29 non-null     object 
 10  int_thrown_away             29 non-null     object 
 11  rushing_away                29 non-null     object 
 12  rushing_att_away            29 non-null     object 
 13  y

## prepare historical file with all needed data points

In [92]:
df_clean_hist = pd.read_csv('data/nfl_historical_clean.csv')
df_clean_hist = df_clean_hist.drop(columns = 'Unnamed: 0')
df_clean_hist = df_clean_hist.append(df_clean, ignore_index=True)
print(df_clean_hist.shape)
df_clean_hist.tail()

(5148, 52)


,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_home_occur,penalty_home_yards,target,winning_team,url
5143,14,7,3,4,55,...,7,47,1,JAX,NaN
5144,22,12,7,3,69,...,5,81,0,PIT,NaN
5145,17,11,6,0,56,...,6,40,0,SEA,NaN
5146,22,10,12,0,72,...,7,60,1,DEN,NaN
5147,11,1,10,0,49,...,3,20,1,NE,NaN


In [96]:
df_clean_hist.to_csv('data/nfl_historical_clean.csv')
df_clean_hist.shape

(5148, 52)

## prepare dataset for model training

In [98]:
df_ready_for_model = df_ready_to_clean.drop(columns=['red_zone_att_away', 'red_zone_att_home', 'url', 'winning_team', 'away_team', 'away_score', 'home_team', 'home_score', 'id'])
#df_ready_for_model.to_csv('data/df_ready_for_model.csv')
df_ready_for_model.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_away_occur,penalty_away_yards,penalty_home_occur,penalty_home_yards,target
0,21,18,3,0,69,...,5,50,10,67,1
1,28,14,7,7,77,...,14,110,14,166,1
2,21,11,9,1,62,...,4,35,4,24,0
3,16,15,1,0,59,...,4,30,3,18,0
4,28,11,13,4,63,...,4,44,6,66,1


In [102]:
df_ready_for_model_hist = pd.read_csv('data/df_ready_for_model.csv')
df_ready_for_model_hist = df_ready_for_model_hist.drop(columns = 'Unnamed: 0')
df_ready_for_model_hist = df_ready_for_model_hist.append(df_ready_for_model, ignore_index=True)
print(df_ready_for_model_hist.shape)
df_ready_for_model_hist.tail()

(5148, 45)


,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_away_occur,penalty_away_yards,penalty_home_occur,penalty_home_yards,target
5143,14,7,3,4,55,...,3,39,7,47,1
5144,22,12,7,3,69,...,10,55,5,81,0
5145,17,11,6,0,56,...,10,86,6,40,0
5146,22,10,12,0,72,...,5,37,7,60,1
5147,11,1,10,0,49,...,6,47,3,20,1


In [103]:
df_ready_for_model_hist.to_csv('data/df_ready_for_model.csv')
df_ready_for_model_hist.shape

(5148, 45)

## re-train model and save

In [106]:
#df_model = pd.read_csv('data/df_ready_for_model.csv')
df_model = df_ready_for_model_hist
df_model['target_class']  = np.where(df_model['target'] == 1, "away", "home")
print(df_model.groupby('target_class').size())
df_model = df_model.drop(columns = ['target'])
df_model = df_model.drop_duplicates()
#df_model

target_class
away    2236
home    2912
dtype: int64


In [107]:
data_bucket = df_model.copy()
data_bucket = data_bucket.dropna()
data_len = len(data_bucket.columns)

y_adj_bucket = data_bucket.iloc[:,data_len-1:]
X_adj_bucket = data_bucket.iloc[:,:data_len-1]

## Random Forest Model

In [108]:
rf_clf =RandomForestClassifier(max_depth = 6,n_estimators=10, max_features = 'log2', random_state=0)
#rf_clf =RandomForestClassifier(max_depth = 6,n_estimators=10, max_features = 'log2', random_state=0, class_weight={'0-4%': 1, '4-7.6%': 2})
np.random.seed(42)

In [112]:
#x_train, x_test, y_train, y_test = train_test_split(X_adj_bucket, y_adj_bucket, shuffle=True, random_state=0)

##smote
sm = SMOTE(random_state=42)

## training
#X_train_res, y_train_res = sm.fit_resample(x_train, y_train)

## productionize
X_res, y_res = sm.fit_resample(X_adj_bucket, y_adj_bucket)

##random oversample
#ros = RandomOverSampler(random_state=42)
#X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

classifier = rf_clf.fit(X_res, y_res)
#classifier = rf_clf.fit(x_train, y_train)
#print("Model Score is:", rf_clf.score(x_test, y_test))
#y_pred_train = rf_clf.predict(X_train_res)
y_pred = rf_clf.predict(X_res)
#y_pred_test = rf_clf.predict(x_test)

## saving model
# save the model to disk
filename = 'models/random_forest_model_1.sav'
pickle.dump(classifier, open(filename, 'wb'))

# load the model from disk
#rf_clf = pickle.load(open(filename, 'rb'))

# View the classification report for test data and predictions
#print("Training Classification Metrics")
#print(classification_report(X_res, y_pred))
#print(classification_report(y_train, y_pred_train))
#print("Testing Classification Metrics")
#print(classification_report(y_test, y_pred_test))

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


## produce team stats aggregation

In [122]:
stats_agg_df = pd.read_csv('data/nfl_historical_clean.csv')
stats_agg_df = stats_agg_df.drop(columns = 'Unnamed: 0')
stats_agg_df = stats_agg_df.drop_duplicates()
stats_agg_df['year'] = stats_agg_df['url'].str[-17:-13]
stats_agg_df['year'] = stats_agg_df['year'].astype(int)
today = date.today()
year = today.year
stats_agg_df = stats_agg_df[stats_agg_df['year'] == year]
stats_agg_df.tail()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,total_plays_away,...,penalty_home_yards,target,winning_team,url,year
5143,14.0,7.0,3.0,4.0,55.0,...,47.0,1,JAX,https://www.espn.com/nfl/scoreboard/_/week/13/...,2021
5144,22.0,12.0,7.0,3.0,69.0,...,81.0,0,PIT,https://www.espn.com/nfl/scoreboard/_/week/13/...,2021
5145,17.0,11.0,6.0,0.0,56.0,...,40.0,0,SEA,https://www.espn.com/nfl/scoreboard/_/week/13/...,2021
5146,22.0,10.0,12.0,0.0,72.0,...,60.0,1,DEN,https://www.espn.com/nfl/scoreboard/_/week/13/...,2021
5147,11.0,1.0,10.0,0.0,49.0,...,20.0,1,NE,https://www.espn.com/nfl/scoreboard/_/week/13/...,2021


In [123]:
team_list = stats_agg_df['away_team'].unique()
len(team_list)

32

In [124]:
stat_agg_df = pd.DataFrame()

for i in team_list:
    team = i

    new_DataFrame = stats_agg_df[(stats_agg_df['away_team'] == team) | (stats_agg_df['home_team'] == team)]

    stat_df_away = new_DataFrame[new_DataFrame['away_team'] == team]
    stat_df_away = stat_df_away[['first_downs_away', 'first_downs_passing_away', 'first_down_rushing_away', 'first_down_by_penalty_away', 'total_plays_away', 'total_yards_away', 'total_drives_away', 'yards_per_play_away', 'passing_away', 'yards_per_pass_away', 'int_thrown_away', 'rushing_away', 'rushing_att_away', 'yards_per_rush_away', 'turnovers_away', 'fumbles_lost_away', 'defensive_td_away', 'possession_away', 'sack_yards_lost_away_occur', 'sack_yards_lost_away_yards', 'penalty_away_occur', 'penalty_away_yards']]
    stat_df_away = stat_df_away.rename(columns = {'first_downs_away': 'first_downs', 'first_downs_passing_away': 'first_downs_passing', 'first_down_rushing_away': 'first_down_rushing', 'first_down_by_penalty_away': 'first_down_by_penalty', 'total_plays_away': 'total_plays', 'total_yards_away': 'total_yards', 'total_drives_away': 'total_drives', 'yards_per_play_away': 'yards_per_play', 'passing_away': 'passing', 'yards_per_pass_away': 'yards_per_pass', 'int_thrown_away': 'int_thrown', 'rushing_away': 'rushing', 'rushing_att_away': 'rushing_att', 'yards_per_rush_away': 'yards_per_rush', 'turnovers_away': 'turnovers', 'fumbles_lost_away': 'fumbles_lost', 'defensive_td_away': 'defensive_td', 'possession_away': 'possession', 'sack_yards_lost_away_occur': 'sack_yards_lost_occur', 'sack_yards_lost_away_yards': 'sack_yards_lost_yards', 'penalty_away_occur': 'penalty_occur', 'penalty_away_yards': 'penalty_yards'})
    
    stat_df_home = new_DataFrame[new_DataFrame['home_team'] == team]
    stat_df_home = stat_df_home[['first_downs_home', 'first_downs_passing_home', 'first_down_rushing_home', 'first_down_by_penalty_home', 'total_plays_home', 'total_yards_home', 'total_drives_home', 'yards_per_play_home', 'passing_home', 'yards_per_pass_home', 'int_thrown_home', 'rushing_home', 'rushing_att_home', 'yards_per_rush_home', 'turnovers_home', 'fumbles_lost_home', 'defensive_td_home', 'possession_home', 'sack_yards_lost_home_occur', 'sack_yards_lost_home_yards', 'penalty_home_occur', 'penalty_home_yards']]
    stat_df_home = stat_df_home.rename(columns = {'first_downs_home': 'first_downs', 'first_downs_passing_home': 'first_downs_passing', 'first_down_rushing_home': 'first_down_rushing', 'first_down_by_penalty_home': 'first_down_by_penalty', 'total_plays_home': 'total_plays', 'total_yards_home': 'total_yards', 'total_drives_home': 'total_drives', 'yards_per_play_home': 'yards_per_play', 'passing_home': 'passing', 'yards_per_pass_home': 'yards_per_pass', 'int_thrown_home': 'int_thrown', 'rushing_home': 'rushing', 'rushing_att_home': 'rushing_att', 'yards_per_rush_home': 'yards_per_rush', 'turnovers_home': 'turnovers', 'fumbles_lost_home': 'fumbles_lost', 'defensive_td_home': 'defensive_td', 'possession_home': 'possession', 'sack_yards_lost_home_occur': 'sack_yards_lost_occur', 'sack_yards_lost_home_yards': 'sack_yards_lost_yards', 'penalty_home_occur': 'penalty_occur', 'penalty_home_yards': 'penalty_yards'})

    team_stats_all = stat_df_away.append(stat_df_home)
    team_stats_agg_temp = pd.DataFrame(team_stats_all.mean()).T
    team_stats_agg_temp['team'] = team
    stat_agg_df = stat_agg_df.append(team_stats_agg_temp)
    print(i, "Done")

print("All Done")

DAL Done
PHI Done
PIT Done
MIN Done
SF Done
ARI Done
SEA Done
LAC Done
NYJ Done
JAX Done
CLE Done
MIA Done
GB Done
DEN Done
CHI Done
BAL Done
NYG Done
CIN Done
HOU Done
LAR Done
BUF Done
NE Done
LV Done
NO Done
ATL Done
TEN Done
KC Done
DET Done
CAR Done
WSH Done
IND Done
TB Done
All Done


In [125]:
stat_agg_df.head()

,first_downs,first_downs_passing,first_down_rushing,first_down_by_penalty,total_plays,...,sack_yards_lost_occur,sack_yards_lost_yards,penalty_occur,penalty_yards,team
0,23.153846,14.538462,6.538462,2.076923,68.153846,...,1.615385,9.692308,8.000000,71.615385,DAL
0,21.000000,9.714286,10.000000,1.285714,62.714286,...,1.714286,10.214286,6.428571,52.285714,PHI
0,19.166667,12.416667,4.750000,2.000000,64.583333,...,2.083333,11.750000,6.250000,48.500000,PIT
0,21.333333,13.250000,6.333333,1.750000,66.083333,...,1.500000,10.666667,7.000000,66.416667,MIN
0,21.083333,11.333333,7.666667,2.083333,61.250000,...,1.833333,10.416667,6.083333,65.250000,SF


In [126]:
stat_agg_df.to_csv('data/agg_team_stats.csv')

## Appending Game Id's to master list

In [139]:
game_id_og = pd.read_csv('data/game_id_all.csv')
game_id_og = game_id_og.drop(columns = ['Unnamed: 0'])
previous_weeks_games_df = previous_week_games_file_df
game_id_all = game_id_og.append(previous_weeks_games_df)
game_id_all = game_id_all.dropna()
game_id_all

,id,url
0,220905019,https://www.espn.com/nfl/scoreboard/_/week/1/y...
1,220908002,https://www.espn.com/nfl/scoreboard/_/week/1/y...
2,220908003,https://www.espn.com/nfl/scoreboard/_/week/1/y...
3,220908004,https://www.espn.com/nfl/scoreboard/_/week/1/y...
4,220908005,https://www.espn.com/nfl/scoreboard/_/week/1/y...
...,...,...
9,401326525,https://www.espn.com/nfl/scoreboard/_/week/14/...
10,401326534,https://www.espn.com/nfl/scoreboard/_/week/14/...
11,401326535,https://www.espn.com/nfl/scoreboard/_/week/14/...
12,401326523,https://www.espn.com/nfl/scoreboard/_/week/14/...


In [140]:
game_scores_og = pd.read_csv('data/game_scores.csv')
game_scores_og = game_scores_og.drop(columns = ['Unnamed: 0'])
game_scores_all = game_scores_og.append(game_score_big_df)
game_scores_all

,away_team,away_score,home_team,home_score,id
0,SF,16,NYG,13,220905019
1,NYJ,37,BUF,31,220908002
2,MIN,23,CHI,27,220908003
3,SD,34,CIN,6,220908004
4,KC,40,CLE,39,220908005
...,...,...,...,...,...
0,JAX,7,LAR,37,401326519
0,BAL,19,PIT,20,401326520
0,SF,23,SEA,30,401326521
0,DEN,9,KC,22,401326515


In [141]:
game_scores_all.to_csv('data/game_scores.csv')

In [142]:
game_stats_og = pd.read_csv('data/game_stats_all.csv')
game_stats_og = game_stats_og.drop(columns = ['Unnamed: 0'])
#game_stats_all = game_stats_og.append(game_stats_big_df)
game_stats_og.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   first_downs_away            4999 non-null   int64  
 1   first_downs_passing_away    4999 non-null   int64  
 2   first_down_rushing_away     4999 non-null   int64  
 3   first_down_by_penalty_away  4999 non-null   int64  
 4   third_down_away             4999 non-null   object 
 5   fourth_down_away            4999 non-null   object 
 6   total_plays_away            4999 non-null   int64  
 7   total_yards_away            4999 non-null   int64  
 8   total_drives_away           4999 non-null   int64  
 9   yards_per_play_away         4999 non-null   float64
 10  passing_away                4999 non-null   int64  
 11  comp_att_away               4999 non-null   object 
 12  yards_per_pass_away         4999 non-null   float64
 13  int_thrown_away             4999 

In [39]:
game_stats_og.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   first_downs_away            4999 non-null   int64  
 1   first_downs_passing_away    4999 non-null   int64  
 2   first_down_rushing_away     4999 non-null   int64  
 3   first_down_by_penalty_away  4999 non-null   int64  
 4   third_down_away             4999 non-null   object 
 5   fourth_down_away            4999 non-null   object 
 6   total_plays_away            4999 non-null   int64  
 7   total_yards_away            4999 non-null   int64  
 8   total_drives_away           4999 non-null   int64  
 9   yards_per_play_away         4999 non-null   float64
 10  passing_away                4999 non-null   int64  
 11  comp_att_away               4999 non-null   object 
 12  yards_per_pass_away         4999 non-null   float64
 13  int_thrown_away             4999 

## update model training data and re-train model.